**CSCI4050U - Machine Learning - Final Project**

Muhammad Jibran Khan - 100877086

**Problem Formulation:**

We study whether latent “fighter style” embeddings, learned from round-level performance statistics, improve the prediction of UFC fight outcomes and methods of victory.

Using fight_performance, round_stats, strike_breakdown, and strike_stats, we construct per-fight vectors describing a fighter’s striking and grappling behavior (e.g., knockdowns, submission attempts, control time, and detailed strike distributions). For each fighter, we order their fights by event.date (via fight and event) and feed the last N fights into a recurrent encoder to produce a time-dependent style embedding *𝑆_𝑖(𝑡)* before each bout.

For each bout, we combine the two fighters’ style embeddings with metadata from fighters and career_stats (height, reach, stance, career per-minute stats, etc.), and use a Siamese neural network to predict (1) the probability that fighter A wins and (2) a distribution over method of victory (KO/TKO, Submission, Decision), using labels from fight_performance.result and fight.method. All features are computed using only data from fights with event.date strictly earlier than the target bout, and train/validation/test sets are split by date to avoid information leakage.

To quantify the value of style embeddings, we compare a metadata-only baseline and an engineered-stat baseline against our embedding-enhanced model, evaluating accuracy, log loss, Brier score, ROC-AUC, and calibration. This allows us to determine whether style representations learned from per-fight statistics meaningfully improve UFC bout prediction.